In [2]:
import os
from ultralytics import YOLO
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

2024-12-27 17:19:03 - matplotlib - DEBUG - matplotlib data path: /home/juanhp/Documents/SalmonsTRT/salmons/lib/python3.10/site-packages/matplotlib/mpl-data
2024-12-27 17:19:03 - matplotlib - DEBUG - CONFIGDIR=/home/juanhp/.config/matplotlib
2024-12-27 17:19:03 - matplotlib - DEBUG - interactive is False
2024-12-27 17:19:03 - matplotlib - DEBUG - platform is linux
2024-12-27 17:19:03 - matplotlib - DEBUG - CACHEDIR=/home/juanhp/.cache/matplotlib
2024-12-27 17:19:03 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/juanhp/.cache/matplotlib/fontlist-v390.json


In [3]:
# CONSTANTES
BATCH_SIZE = -1
C = 3 # number of channels of the input image
H = 224 # heigh of the input image
W = 224 # width of the input image
current_directory = os.getcwd()

## ONNX: Model conversion

In [3]:
%run onnx_transform.py --weights weights/yolov11salmons.pt --input_shape $BATCH_SIZE $C $H $W

CUDA is available.
YOLOv8l-seg summary (fused): 313 layers, 41,738,835 parameters, 0 gradients, 200.5 GFLOPs
model stride:  32
dynamic:  {'images': {0: 'batch', 2: 'height', 3: 'width'}, 'output0': {0: 'batch', 2: 'anchors'}, 'output1': {0: 'batch', 2: 'mask_height', 3: 'mask_width'}}
fake input:  torch.Size([16, 3, 224, 224])


/home/juanhp/Documents/SalmonsTRT/salmons/lib/python3.10/site-packages/ultralytics/utils/tal.py:338: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


La conversión a ONNX se ha completado exitosamente. El modelo se ha guardado en: /home/juanhp/Documents/SalmonsTRT/weights/yolov11salmons.onnx


## TRT fp32

In [4]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --fp32 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_fp32.engine

[12/27/2024-16:28:48] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[12/27/2024-16:28:48] [TRT] [W] input "images" with shape: (-1, 3, -1, -1) dtype: DataType.FLOAT
[12/27/2024-16:28:48] [TRT] [W] output "output0" with shape: (-1, 37, -1) dtype: DataType.FLOAT
[12/27/2024-16:28:48] [TRT] [W] output "output1" with shape: (-1, 32, -1, -1) dtype: DataType.FLOAT
[12/27/2024-16:31:29] [TRT] [W] Build tensorrt engine finish.
Save in /home/juanhp/Documents/SalmonsTRT/weights/yolov11salmons_fp32.engine


## TRT fp16

In [3]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --fp16 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_fp16.engine

[12/27/2024-16:32:17] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[12/27/2024-16:32:17] [TRT] [W] input "images" with shape: (-1, 3, -1, -1) dtype: DataType.FLOAT
[12/27/2024-16:32:17] [TRT] [W] output "output0" with shape: (-1, 37, -1) dtype: DataType.FLOAT
[12/27/2024-16:32:17] [TRT] [W] output "output1" with shape: (-1, 32, -1, -1) dtype: DataType.FLOAT
[12/27/2024-16:39:13] [TRT] [W] TensorRT encountered issues when converting weights between types and that could affect accuracy.
[12/27/2024-16:39:13] [TRT] [W] If this is not the desired behavior, please modify the weights or retrain with regularization to adjust the magnitude of the weights.
[12/27/2024-16:39:13] [TRT] [W] Check verbose logs for the list of affected weights.
[12/27/2024-16:39:13] [TRT] [W] - 114 weights are affected by this issue: Detected subnormal FP16 values.
[12/27/2024-16:39:13] [TRT]

## TRT int8

In [4]:
%run build_trt.py --weights weights/yolov11salmons.onnx  --int8 --input_shape $BATCH_SIZE $C $H $W --engine_name yolov11salmons_int8.engine

2024-12-27 17:19:18 - utils.engine - INFO - Collecting calibration files from: datasets/img_preprocess/
2024-12-27 17:19:18 - utils.engine - INFO - Number of Calibration Files found: 256
2024-12-27 17:19:19 - utils.engine - INFO - Using calibration cache to save time: outputs/cache/calibration.cache
2024-12-27 17:19:19 - utils.engine - INFO - Using calibration cache to save time: outputs/cache/calibration.cache


[12/27/2024-17:19:18] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[12/27/2024-17:19:18] [TRT] [W] input "images" with shape: (-1, 3, -1, -1) dtype: DataType.FLOAT
[12/27/2024-17:19:18] [TRT] [W] output "output0" with shape: (-1, 37, -1) dtype: DataType.FLOAT
[12/27/2024-17:19:18] [TRT] [W] output "output1" with shape: (-1, 32, -1, -1) dtype: DataType.FLOAT
[12/27/2024-17:19:19] [TRT] [W] Missing scale and zero-point for tensor /model.22/Squeeze_output_0, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[12/27/2024-17:19:19] [TRT] [W] Missing scale and zero-point for tensor /model.22/Squeeze_1_output_0, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[12/27/2024-17:19:19] [TRT] [W] Missing scale and zero-point for tensor /model.22/Squeeze_2_output_0, expect fall back to no

## validate

In [ ]:
model = YOLO('weights/yolov11salmons.engine', task='segment')
metrics = model.val(data='datasets/salmons/salmons.yaml', imgsz=224, task='segment', verbose=False,conf=0.4,device='cuda',batch=1)
print('base model box mAP50: ', metrics.box.map50)
print('base model box mAP50-95: ', metrics.box.map)